# Regression Evaluation

In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydataset
import scipy.stats as stats
import sklearn.metrics
from sklearn.linear_model import LinearRegression

Punchline: is our model better than the model that just predicts the average?

The best model we can make with no additional information: $\hat{y} = \bar{y}$

**Baseline comes from train split!**

## Setup / Functions

In [ ]:
def residuals(actual, predicted):
    return actual - predicted

def sse(actual, predicted):
    return (residuals(actual, predicted) ** 2).sum()

def mse(actual, predicted):
    n = actual.shape[0]
    return sse(actual, predicted) / n

def rmse(actual, predicted):
    return math.sqrt(mse(actual, predicted))

def ess(actual, predicted):
    return ((predicted - actual.mean()) ** 2).sum()

def tss(actual):
    return ((actual - actual.mean()) ** 2).sum()

def r2_score(actual, predicted):
    return ess(actual, predicted) / tss(actual)

In [ ]:
def plot_residuals(actual, predicted):
    residuals = actual - predicted
    plt.hlines(0, actual.min(), actual.max(), ls=':')
    plt.scatter(actual, residuals)
    plt.ylabel('residual ($y - \hat{y}$)')
    plt.xlabel('actual value ($y$)')
    plt.title('Actual vs Residual')
    return plt.gca()

def regression_errors(actual, predicted):
    return pd.Series({
        'sse': sse(actual, predicted),
        'ess': ess(actual, predicted),
        'tss': tss(actual),
        'mse': mse(actual, predicted),
        'rmse': rmse(actual, predicted),
        'r2': r2_score(actual, predicted),
    })

def baseline_mean_errors(actual):
    predicted = actual.mean()
    return {
        'sse': sse(actual, predicted),
        'mse': mse(actual, predicted),
        'rmse': rmse(actual, predicted),
    }

def better_than_baseline(actual, predicted):
    sse_baseline = sse(actual, actual.mean())
    sse_model = sse(actual, predicted)
    return sse_model < sse_baseline

## Use the Functions

### tips

In [ ]:
tips = pydataset.data('tips')
model = LinearRegression().fit(tips[['total_bill']], tips.tip)

In [ ]:
# we can use the fit model on new data
new_data = pd.Series([10, 20], name='total_bill').values.reshape(-1, 1)
model.predict(new_data)

In [ ]:
actual = tips.tip
predicted = model.predict(tips[['total_bill']])

In [ ]:
model2 = LinearRegression().fit(tips[['size']], tips.tip)
predicted2 = model2.predict(tips[['size']])

In [ ]:
regression_errors(actual, predicted2)

In [ ]:
regression_errors(actual, predicted)

In [ ]:
regression_errors(actual, predicted)['sse'] < regression_errors(actual, predicted2)['sse']

In [ ]:
better_than_baseline(actual, predicted)

In [ ]:
pd.DataFrame({
    'model 1 (tip ~ total_bill)': regression_errors(actual, predicted),
    'model 2 (tip ~ size)': regression_errors(actual, predicted2),
    'baseline model': regression_errors(actual, np.repeat(actual.mean(), actual.shape[0]))
})

#### Sidenote: comparing our r2 to sklearn's

In [ ]:
sklearn_r2 = sklearn.metrics.r2_score(actual, predicted)

In [ ]:
our_r2 = r2_score(actual, predicted)

In [ ]:
our_r2 == sklearn_r2

In [ ]:
our_r2, sklearn_r2

In [ ]:
np.isclose(our_r2, sklearn_r2)

In [ ]:
print('our mse', mse(actual, predicted))
print('sklearn', sklearn.metrics.mean_squared_error(actual, predicted))

### mpg

In [ ]:
mpg = pydataset.data('mpg')

In [ ]:
model = LinearRegression().fit(mpg[['displ']], mpg.hwy)

In [ ]:
actual = mpg.hwy # y
predicted = model.predict(mpg[['displ']]) # yhat

In [ ]:
regression_errors(actual, predicted)

In [ ]:
better_than_baseline(actual, predicted)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
ax.scatter(mpg.displ, mpg.hwy, label='actual')
ax.scatter(mpg.displ, predicted, label='prediction')

Sidenote: visualizing residuals w/ multiple independent variables

In [ ]:
plt.scatter(mpg.hwy, actual - predicted)

In [ ]:
plt.hist(actual - predicted, bins=15)